In [1]:
"""Example of using training on CartPole."""

#import argparse
#/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/model_ensemble.py

import ray
from ray import tune
from ray.rllib.agents.mbmpo.model_ensemble import DynamicsEnsembleCustomModel
from ray.rllib.examples.env.mbmpo_env import CartPoleWrapper
from ray.rllib.models.catalog import ModelCatalog
from ray.rllib.agents.mbmpo.utils import MBMPOExploration 
from ray.rllib.env.wrappers.model_vector_env import model_vector_env
from ray.rllib.utils.deprecation import DEPRECATED_VALUE
from ray.rllib.agents.mbmpo.mbmpo import MBMPOTrainer
train=True
#parser = argparse.ArgumentParser()
#parser.add_argument("--num-workers", default=6, type=int)
#parser.add_argument("--training-iteration", default=10000, type=int)
#parser.add_argument("--ray-num-cpus", default=7, type=int)
#args = parser.parse_args()
ray.shutdown()
ray.init(num_cpus=7)

ModelCatalog.register_custom_model("DynamicsEnsembleCustomModel", DynamicsEnsembleCustomModel)

config={
    "env": CartPoleWrapper,
    "use_gae": True,
    # GAE(lambda) parameter.
    "lambda": 1.0,
    # Initial coefficient for KL divergence.
    "kl_coeff": 0.0005,
    # Size of batches collected from each worker.
    "rollout_fragment_length": 200,
    # Do create an actual env on the local worker (worker-idx=0).
    "create_env_on_driver": True,
    # Step size of SGD.
    "lr": 1e-3,
    # Coefficient of the value function loss.
    "vf_loss_coeff": 0.5,
    # Coefficient of the entropy regularizer.
    "entropy_coeff": 0.0,
    # PPO clip parameter.
    "clip_param": 0.5,
    # Clip param for the value function. Note that this is sensitive to the
    # scale of the rewards. If your expected V is large, increase this.
    "vf_clip_param": 10.0,
    # If specified, clip the global norm of gradients by this amount.
    "grad_clip": None,
    # Target value for KL divergence.
    "kl_target": 0.01,
    # Whether to rollout "complete_episodes" or "truncate_episodes".
    "batch_mode": "complete_episodes",
    # Which observation filter to apply to the observation.
    "observation_filter": "NoFilter",
    # Number of Inner adaptation steps for the MAML algorithm.
    "inner_adaptation_steps": 1,
    # Number of MAML steps per meta-update iteration (PPO steps).
    "maml_optimizer_steps": 8,
    # Inner adaptation step size.
    "inner_lr": 1e-3,
    # Horizon of the environment (200 in MB-MPO paper).
    "horizon": 200,
    # Dynamics ensemble hyperparameters.
    "dynamics_model": {
        "custom_model": DynamicsEnsembleCustomModel,
        # Number of Transition-Dynamics (TD) models in the ensemble.
        "ensemble_size": 5,
        # Hidden layers for each model in the TD-model ensemble.
        "fcnet_hiddens": [512, 512, 512],
        # Model learning rate.
        "lr": 1e-3,
        # Max number of training epochs per MBMPO iter.
        "train_epochs": 500,
        # Model batch size.
        "batch_size": 500,
        # Training/validation split.
        "valid_split_ratio": 0.2,
        # Normalize data (obs, action, and deltas).
        "normalize_data": True,
    },
    # Exploration for MB-MPO is based on StochasticSampling, but uses 8000
    # random timesteps up-front for worker=0.
    "exploration_config": {
        "type": MBMPOExploration,
        "random_timesteps": 8000,
    },
    # Workers sample from dynamics models, not from actual envs.
    "custom_vector_env": model_vector_env,
    # How many iterations through MAML per MBMPO iteration.
    "num_maml_steps": 10,

    # Deprecated keys:
    # Share layers for value function. If you set this to True, it's important
    # to tune vf_loss_coeff.
    # Use config.model.vf_share_layers instead.
    "vf_share_layers": DEPRECATED_VALUE,
}

from contextlib import contextmanager
import sys, os
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [2]:
trainer = MBMPOTrainer(
    config=config,
    env="ray.rllib.examples.env.mbmpo_env.CartPoleWrapper")

env = CartPoleWrapper()
#env = gym.make('LunarL')
#env.render()
# run until episode ends
# list that lists all actions
iteration_list=[]
epoch_nr=0
for _ in range(5):
    epoch_nr+=1
    with suppress_stdout():
        trainer.train()
    print(f"trainer trained: {epoch_nr} Epochs")
    trainer.save("/Users/felix/sciebo/masterarbeit/progra/model-based_rl/mbmpo_check")

2022-08-18 10:36:13,592	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-08-18 10:36:13,603	WARNING mbmpo.py:351 -- MB-MPO only supported in PyTorch so far! Switching to `framework=torch`.
2022-08-18 10:36:13,604	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-08-18 10:36:13,695	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-08-18 10:36:14,089	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` 

Training Dynamics Ensemble - Epoch #0:Train loss: 1.0244043 1.0177811 0.958069 0.9996267 1.0046558, Valid Loss: 0.7060777 0.7210353 0.7389786 0.6927346 0.6807207,  Moving Avg Valid Loss: 1.0414646 1.0635271 1.0899935 1.0217835 1.0040629
Training Dynamics Ensemble - Epoch #1:Train loss: 0.73617774 0.7062043 0.666462 0.68393123 0.70411044, Valid Loss: 0.47564933 0.47751516 0.41951814 0.4376904 0.42459798,  Moving Avg Valid Loss: 1.0131738 1.0342264 1.0564697 0.9925788 0.97508967
Training Dynamics Ensemble - Epoch #2:Train loss: 0.47969782 0.42715365 0.4009486 0.40463194 0.43163553, Valid Loss: 0.24128017 0.23920935 0.1461424 0.19061852 0.17496255,  Moving Avg Valid Loss: 0.9745791 0.99447554 1.0109533 0.95248073 0.93508327
Training Dynamics Ensemble - Epoch #3:Train loss: 0.22634193 0.18853109 0.15329848 0.16549137 0.17201005, Valid Loss: 0.08172831 0.06509285 0.060785085 0.038180195 0.036184985,  Moving Avg Valid Loss: 0.9299366 0.9480064 0.9634449 0.9067657 0.8901383
Training Dynamics 

(RolloutWorker pid=92237) 2022-08-18 10:36:20,611	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=92238) 2022-08-18 10:36:20,611	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


Training Dynamics Ensemble - Epoch #81:Train loss: 0.0004141019 0.0004730763 0.00041678973 0.0004386034 0.00040851044, Valid Loss: 0.0011155477 0.0006556312 0.001426249 0.00095005863 0.0005228154,  Moving Avg Valid Loss: 0.019979812 0.019349823 0.020880366 0.019160317 0.018283976
Training Dynamics Ensemble - Epoch #82:Train loss: 0.00042029502 0.00046653402 0.00040059345 0.0004414736 0.00038908617, Valid Loss: 0.0010964075 0.0006518551 0.001434425 0.00095292483 0.00053932174,  Moving Avg Valid Loss: 0.019035641 0.018414924 0.019908069 0.018249948 0.017396742
Training Dynamics Ensemble - Epoch #83:Train loss: 0.000426437 0.00045715805 0.00038110162 0.00043967765 0.00038452464, Valid Loss: 0.0010850073 0.00065231795 0.0014225678 0.00095474906 0.0005673046,  Moving Avg Valid Loss: 0.018138109 0.017526794 0.018983794 0.017385188 0.01655527
Training Dynamics Ensemble - Epoch #84:Train loss: 0.00042267307 0.00044385766 0.00037109785 0.00042914768 0.00039109113, Valid Loss: 0.0010795647 0.000

2022-08-18 10:36:43,849	INFO trainable.py:159 -- Trainable.setup took 30.261 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-08-18 10:36:43,851	WARNING util.py:65 -- Install gputil for GPU system monitoring.
/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self._coeffs = np.linalg.lstsq(
/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence th

...

trainer trained: 3 Epochs


/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self._coeffs = np.linalg.lstsq(
/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self._coeffs = np.linalg.lstsq(
/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the d

trainer trained: 4 Epochs


/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self._coeffs = np.linalg.lstsq(
/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self._coeffs = np.linalg.lstsq(
/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/agents/mbmpo/utils.py:36: FutureWarning: `rcond` parameter will change to the d

trainer trained: 5 Epochs


In [12]:
env = CartPoleWrapper()
episode_reward = 0
done = False
obs = env.reset()
while not done:
    action = trainer.compute_action(obs)
    obs, reward, done, info = env.step(action)
    episode_reward += reward
    env.render()
env.close()
print(episode_reward)

46.0


In [ ]:
if train:
    analysis=tune.run(
        "MBMPO",
        local_dir="mbmpo_check",
        stop={"training_iteration": 1},
        checkpoint_freq = 1,
        max_failures=0,
        config=config,
    )



In [ ]:
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.policy.policy_map import PolicyMap
from ray.rllib.evaluation.episode import MultiAgentEpisode

agent = AlphaZeroTrainer (env = CartPole, config = config)
env=CartPole()
agent.restore ('tune_check/contrib/AlphaZero/contrib_AlphaZero_CartPole_dede1_00000_0_2022-08-17_20-46-14/checkpoint_000040/checkpoint-40')

reward_list=[]
for i in range(20):
    obs = env.reset()
    policy = agent.get_policy(DEFAULT_POLICY_ID)
    episode = MultiAgentEpisode(
        PolicyMap(0,0),
        lambda _, __: DEFAULT_POLICY_ID,
        lambda: None,
        lambda _: None,
        0,
    )

    episode.user_data['initial_state'] = env.get_state()

    done = False

    while not done:
        action, _, _ = policy.compute_single_action(obs, episode=episode)
        obs, reward, done, _ = env.step(action)
        #print(reward)
        #env.render()
        #time.sleep(0.1)
        episode.length += 1
        
    reward_list.append(reward)
    print(f"episode {i} finished")
#env.close()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(reward_list,'bs')
plt.ylabel('gained reward')
plt.xlabel('runs')
plt.title('AlphaZero reward in Cartpole Environment over different runs checkpoint_40')
plt.show()